<a href="https://colab.research.google.com/github/AlessiaGouveia/CSV-to-STL/blob/main/CSV_to_STL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Function to read x, y, z coordinates from a CSV file
def load_xyz_from_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        x = df.iloc[:, 0].values  # First column (x)
        y = df.iloc[:, 1].values  # Second column (y)
        z = df.iloc[:, 2].values  # Third column (z)

        # Verify if any column contains missing values and remove these rows.
        missing_value_rows = df[df.isnull().any(axis=1)].index
        if len(missing_value_rows) > 0:
            print(f"Warning: Found {len(missing_value_rows)} rows with missing values. Removing these rows for proper processing.")
            df.dropna(inplace=True)
        x = df.iloc[:, 0].values  # First column (x)
        y = df.iloc[:, 1].values  # Second column (y)
        z = df.iloc[:, 2].values  # Third column (z)
        return x, y, z
    except FileNotFoundError:
        print(f"Error: File not found at path {file_path}")
        return None, None, None
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None, None, None

# Function to calculate the normal vector of a triangle
def calculate_normal(triangle):
    v1 = triangle[1] - triangle[0]
    v2 = triangle[2] - triangle[0]
    normal = np.cross(v1, v2)
    norm = np.linalg.norm(normal)
    if norm == 0:
        return np.array([0, 0, 0])  # Return zero vector if triangle is degenerate
    return normal / norm

# Function to write the STL file
def write_stl_triangle(file, v1, v2, v3, normal):
    file.write('  facet normal {} {} {}\n'.format(*normal))
    file.write('    outer loop\n')
    file.write('      vertex {} {} {}\n'.format(*v1))
    file.write('      vertex {} {} {}\n'.format(*v2))
    file.write('      vertex {} {} {}\n'.format(*v3))
    file.write('    endloop\n')
    file.write('  endfacet\n')

def write_stl_file(file_path, points, triangulation):
    with open(file_path, 'w') as f:
        f.write('solid STL_from_CSV\n')
        for simplex in triangulation.simplices:
            triangle = points[simplex]
            normal = calculate_normal(triangle)
            write_stl_triangle(f, triangle[0], triangle[1], triangle[2], normal)
        f.write('endsolid STL_from_CSV\n')


# Load x, y, z coordinates from the CSV file
x, y, z = load_xyz_from_csv(csv_name)

# Check if the data loading was successful
if x is None or y is None or z is None:
    print("Failed to load data from the CSV file.")
    exit()

# Combine x, y, and z into a single array of 3D points
points = np.column_stack((x, y, z))

# Triangulate the points for the surface plot (using x and y coordinates)
points_2d = np.column_stack((x, y))
try:
    tri = Delaunay(points_2d)
except Exception as e:
    print(f"Error during Delaunay triangulation or plotting: {e}")
    print("Check if the points are co-linear or form a degenerate configuration.")
    exit()

# Create STL name
stl_name = "CSV_to_STL.stl"

# Generate STL file
stl_file_path = stl_name
try:
    write_stl_file(stl_file_path, points, tri)
    print(f"STL file saved to {stl_file_path}")
except Exception as e:
    print(f"Error writing STL file: {e}")